# Initialize

In [1]:
import csv
import pickle
import pandas as pd
import numpy as np

# Load data

In [2]:
file = open('lightcurves_1','rb')
lightcurves = pickle.load(file)

In [3]:
print(lightcurves['Data'][1]['R_mag'])
print(lightcurves['Data'][1])

[19.226376 19.249918 19.175348 19.543024 19.699879 19.713348 19.84064
 19.820326]
{'R_mag': array([19.226376, 19.249918, 19.175348, 19.543024, 19.699879, 19.713348,
       19.84064 , 19.820326]), 'R_err': array([0.09768647, 0.1129363 , 0.13859841, 0.10954028, 0.08643299,
       0.09358331, 0.10791072, 0.1002604 ]), 'R_mjd': array([59501.4163426, 59503.2210648, 59505.2      , 59517.1979051,
       59520.3061227, 59522.2476157, 59524.3264815, 59526.2247801]), 'R_non': {'mag': array([20.3956, 19.8101, 19.1459, 19.5908, 18.7131, 20.7832, 20.7587,
       20.3072, 19.9058]), 'mjd': array([59474.3731713, 59476.2839699, 59478.342662 , 59480.3497569,
       59485.3014699, 59488.2629282, 59490.349213 , 59497.371875 ,
       59510.2577199])}, 'G_mag': array([19.210064, 18.993626, 19.06784 , 19.197407, 19.161198, 19.275217,
       19.272158]), 'G_err': array([0.15684326, 0.12696342, 0.09707339, 0.07324645, 0.07865738,
       0.08261283, 0.0817621 ]), 'G_mjd': array([59503.2542824, 59505.2445486, 5

# Put in non-detections
We are interested in all non-detections that occur after the first detection and which are within 3 magnitudes of the closest point + the last two non-detections before the start

In [4]:
# Defined parameters
magdiff = 2 # Define the maximum difference a non-detection can have compared to the point before
timediff = 14 # In days# Define the time before the first detection where non-detections are included
#magerror = 1 # The added error in magnitude for each non-detection

# Listing the lists, so that they work later
R_non_mjd_bool = []
G_non_mjd_bool = []
R_non_mjd2_bool = []
G_non_mjd2_bool = []
R_non_mag_bool = []
G_non_mag_bool = []

R_non_mjd_include = []
R_non_mag_include = []
R_non_mjd_additional = []
R_non_mag_additional = []
G_non_mjd_include = []
G_non_mag_include = []
G_non_mjd_additional = []
G_non_mag_additional = []

R_non_mjd_use = []
R_non_mag_use = []
G_non_mjd_use = []
G_non_mag_use = []


# Loop start to work with all of them
for i in range(len(lightcurves['Data'])): # Go through each light curve
#for i in range(400):
    # Save old non-detections
    lightcurves['Data'][i]['R_non_org'] = lightcurves['Data'][i]['R_non']
    lightcurves['Data'][i]['G_non_org'] = lightcurves['Data'][i]['G_non']
    
    ## R: Red Dead Redemption Edition
    
    # Remove all non-detections that have occured at the same time as a real detection
    if type(lightcurves['Data'][i]['R_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
        R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection(lightcurves['Data'][i]['R_non']['mjd'])) # Then find the intersection between non-detections and detections
        
        if len(R_non_set) > 0: # if it is non-empty
            lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            
    else:
        R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection([lightcurves['Data'][i]['R_non']['mjd']]))
        
        if len(R_non_set) > 0:
            lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
    
    # Check if code has put in a -1:
    
    if type(lightcurves['Data'][i]['R_non']['mjd']) == int:
        # Give all of them an empty array so that it works
        R_non_mjd_bool.append([float('nan')])
        R_non_mjd_include.append([float('nan')])
        R_non_mag_include.append([float('nan')])
        R_non_mag_bool.append([float('nan')])
        R_non_mag_use.append([float('nan')])
        R_non_mjd_use.append([float('nan')])
        R_non_mjd2_bool.append([float('nan')])
        R_non_mjd_additional.append([float('nan')])
        R_non_mag_additional.append([float('nan')])
        
        # Put in the actual lightcurves
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag']
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd']
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err']
        
    else:
        # Start date
        R_start = lightcurves['Data'][i]['R_mjd'][0] # Find the start date

        # Find the non-detections that lies timediff before we start
        R_non_mjd2_bool.append((lightcurves['Data'][i]['R_non']['mjd'] > R_start - timediff) & (lightcurves['Data'][i]['R_non']['mjd'] < R_start)) # For those where we don't need to check for magdiff

        R_non_mjd_additional.append(lightcurves['Data'][i]['R_non']['mjd'][R_non_mjd2_bool[i]])
        R_non_mag_additional.append(lightcurves['Data'][i]['R_non']['mag'][R_non_mjd2_bool[i]])


        # The new data vectors are added to the existing vectors

        lightcurves['Data'][i]['R_mag_wn'] = np.append(lightcurves['Data'][i]['R_mag'],R_non_mag_additional[i])
        lightcurves['Data'][i]['R_mjd_wn'] = np.append(lightcurves['Data'][i]['R_mjd'],R_non_mjd_additional[i])

        # Remove NaNs
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag_wn'][~np.isnan(lightcurves['Data'][i]['R_mag_wn'])] # Removes NaNs
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd_wn'][~np.isnan(lightcurves['Data'][i]['R_mjd_wn'])]

        # A new error list is added as well, with magerror being put in
        lightcurves['Data'][i]['R_err_wn'] = np.append(lightcurves['Data'][i]['R_err'],R_non_mag_additional[i])
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err_wn'][~np.isnan(lightcurves['Data'][i]['R_err_wn'])] # remove NaN again

        # Sort the list so that they follow in time
        R_sort = np.argsort(lightcurves['Data'][i]['R_mjd_wn']) # argsort gives indices for the correct order
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag_wn'][R_sort]
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd_wn'][R_sort]
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err_wn'][R_sort]
    
    ## G: Green New Deal Edition
    # Remove all non-detections that have occured at the same time as a real detection
    if type(lightcurves['Data'][i]['G_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
        G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection(lightcurves['Data'][i]['G_non']['mjd'])) # Then find the intersection between non-detections and detections
        
        if len(G_non_set) > 0: # if it is non-empty
            lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            
    else:
        G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection([lightcurves['Data'][i]['G_non']['mjd']]))
        
        if len(G_non_set) > 0:
            lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
    
    # Check if code has put in a -1:
    
    if type(lightcurves['Data'][i]['G_non']['mjd']) == int:
        # Give all of them an empty array so that it works
        G_non_mjd_bool.append([float('nan')])
        G_non_mjd_include.append([float('nan')])
        G_non_mag_include.append([float('nan')])
        G_non_mag_bool.append([float('nan')])
        G_non_mag_use.append([float('nan')])
        G_non_mjd_use.append([float('nan')])
        G_non_mjd2_bool.append([float('nan')])
        G_non_mjd_additional.append([float('nan')])
        G_non_mag_additional.append([float('nan')])
        
        # Put in the actual lightcurves
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag']
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd']
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err']
        
    else:
        # Start date
        G_start = lightcurves['Data'][i]['G_mjd'][0] # Find the start date

        # Find the non-detections that lies timediff before we start
        G_non_mjd2_bool.append((lightcurves['Data'][i]['G_non']['mjd'] > G_start - timediff) & (lightcurves['Data'][i]['G_non']['mjd'] < G_start)) # For those where we don't need to check for magdiff

        G_non_mjd_additional.append(lightcurves['Data'][i]['G_non']['mjd'][G_non_mjd2_bool[i]])
        G_non_mag_additional.append(lightcurves['Data'][i]['G_non']['mag'][G_non_mjd2_bool[i]])


        # The new data vectors are added to the existing vectors

        lightcurves['Data'][i]['G_mag_wn'] = np.append(lightcurves['Data'][i]['G_mag'],G_non_mag_additional[i])
        lightcurves['Data'][i]['G_mjd_wn'] = np.append(lightcurves['Data'][i]['G_mjd'],G_non_mjd_additional[i])

        # Remove NaNs
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag_wn'][~np.isnan(lightcurves['Data'][i]['G_mag_wn'])] # Removes NaNs
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd_wn'][~np.isnan(lightcurves['Data'][i]['G_mjd_wn'])]

        # A new error list is added as well, with magerror being put in
        lightcurves['Data'][i]['G_err_wn'] = np.append(lightcurves['Data'][i]['G_err'],np.append(G_non_mag_additional[i],G_non_mag_use[i]))
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err_wn'][~np.isnan(lightcurves['Data'][i]['G_err_wn'])] # remove NaN again
        
        # Sort the list so that they follow in time
        G_sort = np.argsort(lightcurves['Data'][i]['G_mjd_wn']) # argsort gives indices for the correct order
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag_wn'][G_sort]
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd_wn'][G_sort]
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err_wn'][G_sort]

/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:137: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/Users/davidivens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:138: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [ ]:
## Not used - would be used for 

# Get the last two non-detections - as long as they have come within two weeks
#    Falseplaces[i] = R_non_mjd_bool[i].where(R_non_mjd_bool[i] == False)
#if len(Falseplaces[i]) - sum(Falseplaces[i]) >= 2:
#        
#        
#    elif len(Falseplaces[i]) - sum(Falseplaces[i]) == 1:
#    
#    else:
#        continue

In [ ]:
def detpredet(lightcurves,magdiff,timediff,magerror):
    # Listing the lists, so that they work later
R_non_mjd_bool = []
G_non_mjd_bool = []
R_non_mjd2_bool = []
G_non_mjd2_bool = []
R_non_mag_bool = []
G_non_mag_bool = []

R_non_mjd_include = []
R_non_mag_include = []
R_non_mjd_additional = []
R_non_mag_additional = []
G_non_mjd_include = []
G_non_mag_include = []
G_non_mjd_additional = []
G_non_mag_additional = []

R_non_mjd_use = []
R_non_mag_use = []
G_non_mjd_use = []
G_non_mag_use = []


# Loop start to work with all of them
for i in range(len(lightcurves['Data'])): # Go through each light curve
#for i in range(400):
    # Save old non-detections
    lightcurves['Data'][i]['R_non_org'] = lightcurves['Data'][i]['R_non']
    lightcurves['Data'][i]['G_non_org'] = lightcurves['Data'][i]['G_non']
    
    ## R: Red Dead Redemption Edition
    
    # Remove all non-detections that have occured at the same time as a real detection
    if type(lightcurves['Data'][i]['R_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
        R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection(lightcurves['Data'][i]['R_non']['mjd'])) # Then find the intersection between non-detections and detections
        
        if len(R_non_set) > 0: # if it is non-empty
            lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            
    else:
        R_non_set = list(set(lightcurves['Data'][i]['R_mjd']).intersection([lightcurves['Data'][i]['R_non']['mjd']]))
        
        if len(R_non_set) > 0:
            lightcurves['Data'][i]['R_non']['mag'] = lightcurves['Data'][i]['R_non']['mag'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
            lightcurves['Data'][i]['R_non']['mjd'] = lightcurves['Data'][i]['R_non']['mjd'][~(lightcurves['Data'][i]['R_non']['mjd']==R_non_set)]
    
    # Check if code has put in a -1:
    
    if type(lightcurves['Data'][i]['R_non']['mjd']) == int:
        # Give all of them an empty array so that it works
        R_non_mjd_bool.append([float('nan')])
        R_non_mjd_include.append([float('nan')])
        R_non_mag_include.append([float('nan')])
        R_non_mag_bool.append([float('nan')])
        R_non_mag_use.append([float('nan')])
        R_non_mjd_use.append([float('nan')])
        R_non_mjd2_bool.append([float('nan')])
        R_non_mjd_additional.append([float('nan')])
        R_non_mag_additional.append([float('nan')])
        
        # Put in the actual lightcurves
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag']
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd']
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err']
        
    else:
        # Start date
        R_start = lightcurves['Data'][i]['R_mjd'][0] # Find the start date

        # Find the non-detections that lies timediff before we start
        R_non_mjd2_bool.append((lightcurves['Data'][i]['R_non']['mjd'] > R_start - timediff) & (lightcurves['Data'][i]['R_non']['mjd'] < R_start)) # For those where we don't need to check for magdiff

        R_non_mjd_additional.append(lightcurves['Data'][i]['R_non']['mjd'][R_non_mjd2_bool[i]])
        R_non_mag_additional.append(lightcurves['Data'][i]['R_non']['mag'][R_non_mjd2_bool[i]])


        # The new data vectors are added to the existing vectors

        lightcurves['Data'][i]['R_mag_wn'] = np.append(lightcurves['Data'][i]['R_mag'],R_non_mag_additional[i])
        lightcurves['Data'][i]['R_mjd_wn'] = np.append(lightcurves['Data'][i]['R_mjd'],R_non_mjd_additional[i])

        # Remove NaNs
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag_wn'][~np.isnan(lightcurves['Data'][i]['R_mag_wn'])] # Removes NaNs
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd_wn'][~np.isnan(lightcurves['Data'][i]['R_mjd_wn'])]

        # A new error list is added as well, with magerror being put in
        lightcurves['Data'][i]['R_err_wn'] = np.append(lightcurves['Data'][i]['R_err'],R_non_mag_additional[i])
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err_wn'][~np.isnan(lightcurves['Data'][i]['R_err_wn'])] # remove NaN again

        # Sort the list so that they follow in time
        R_sort = np.argsort(lightcurves['Data'][i]['R_mjd_wn']) # argsort gives indices for the correct order
        lightcurves['Data'][i]['R_mag_wn'] = lightcurves['Data'][i]['R_mag_wn'][R_sort]
        lightcurves['Data'][i]['R_mjd_wn'] = lightcurves['Data'][i]['R_mjd_wn'][R_sort]
        lightcurves['Data'][i]['R_err_wn'] = lightcurves['Data'][i]['R_err_wn'][R_sort]
    
    ## G: Green New Deal Edition
    # Remove all non-detections that have occured at the same time as a real detection
    if type(lightcurves['Data'][i]['G_non']['mjd']) == np.ndarray: # Check if it is an array = has multipl evalue
        G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection(lightcurves['Data'][i]['G_non']['mjd'])) # Then find the intersection between non-detections and detections
        
        if len(G_non_set) > 0: # if it is non-empty
            lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            
    else:
        G_non_set = list(set(lightcurves['Data'][i]['G_mjd']).intersection([lightcurves['Data'][i]['G_non']['mjd']]))
        
        if len(G_non_set) > 0:
            lightcurves['Data'][i]['G_non']['mag'] = lightcurves['Data'][i]['G_non']['mag'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
            lightcurves['Data'][i]['G_non']['mjd'] = lightcurves['Data'][i]['G_non']['mjd'][~(lightcurves['Data'][i]['G_non']['mjd']==G_non_set)]
    
    # Check if code has put in a -1:
    
    if type(lightcurves['Data'][i]['G_non']['mjd']) == int:
        # Give all of them an empty array so that it works
        G_non_mjd_bool.append([float('nan')])
        G_non_mjd_include.append([float('nan')])
        G_non_mag_include.append([float('nan')])
        G_non_mag_bool.append([float('nan')])
        G_non_mag_use.append([float('nan')])
        G_non_mjd_use.append([float('nan')])
        G_non_mjd2_bool.append([float('nan')])
        G_non_mjd_additional.append([float('nan')])
        G_non_mag_additional.append([float('nan')])
        
        # Put in the actual lightcurves
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag']
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd']
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err']
        
    else:
        # Start date
        G_start = lightcurves['Data'][i]['G_mjd'][0] # Find the start date

        # Find the non-detections that lies timediff before we start
        G_non_mjd2_bool.append((lightcurves['Data'][i]['G_non']['mjd'] > G_start - timediff) & (lightcurves['Data'][i]['G_non']['mjd'] < G_start)) # For those where we don't need to check for magdiff

        G_non_mjd_additional.append(lightcurves['Data'][i]['G_non']['mjd'][G_non_mjd2_bool[i]])
        G_non_mag_additional.append(lightcurves['Data'][i]['G_non']['mag'][G_non_mjd2_bool[i]])


        # The new data vectors are added to the existing vectors

        lightcurves['Data'][i]['G_mag_wn'] = np.append(lightcurves['Data'][i]['G_mag'],G_non_mag_additional[i])
        lightcurves['Data'][i]['G_mjd_wn'] = np.append(lightcurves['Data'][i]['G_mjd'],G_non_mjd_additional[i])

        # Remove NaNs
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag_wn'][~np.isnan(lightcurves['Data'][i]['G_mag_wn'])] # Removes NaNs
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd_wn'][~np.isnan(lightcurves['Data'][i]['G_mjd_wn'])]

        # A new error list is added as well, with magerror being put in
        lightcurves['Data'][i]['G_err_wn'] = np.append(lightcurves['Data'][i]['G_err'],np.append(G_non_mag_additional[i],G_non_mag_use[i]))
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err_wn'][~np.isnan(lightcurves['Data'][i]['G_err_wn'])] # remove NaN again
        
        # Sort the list so that they follow in time
        G_sort = np.argsort(lightcurves['Data'][i]['G_mjd_wn']) # argsort gives indices for the correct order
        lightcurves['Data'][i]['G_mag_wn'] = lightcurves['Data'][i]['G_mag_wn'][G_sort]
        lightcurves['Data'][i]['G_mjd_wn'] = lightcurves['Data'][i]['G_mjd_wn'][G_sort]
        lightcurves['Data'][i]['G_err_wn'] = lightcurves['Data'][i]['G_err_wn'][G_sort]
        
    return lightcurves